In [16]:
import requests
from bs4 import BeautifulSoup

url = "http://historical.elections.virginia.gov/elections/search/year_from:1924/year_to:2015/office_id:1/stage:General" #
req = requests.get(url)
html = req.content
soup = BeautifulSoup(html, 'html.parser')

with open("ELECTION_ID", "w") as out:
    for result in soup.find_all("tr", "election_item"):
        year = result.find("td", "year first").contents[0]
        id_number = result["id"].split("-")[2]
        out.write("{} {}\n".format(year, id_number))


In [26]:
import requests
from bs4 import BeautifulSoup

for line in open("ELECTION_ID"):
    year = line[:4]
    code = line.split()[-1]

    data = "http://historical.elections.virginia.gov/elections/download/{}/precincts_include:0/".format(code)
    resp = requests.get(data)
    soup = BeautifulSoup(resp.text,"html.parser")


    file_name = year+".csv"
    with open(file_name,"w") as out:
        out.write(resp.text)


In [46]:
from bs4 import BeautifulSoup
import sys
import requests
import pandas as pd

df_list = []
for x in range(1924,2013,4):
    x = str(x)
    file_name=x+".csv"
    header = pd.read_csv(file_name, nrows=1).dropna(axis=1)
    d = header.iloc[0].to_dict()

    df = pd.read_csv(file_name, index_col=0,thousands=",", skiprows=[1])

    df.rename(inplace = True, columns = d)
    df.dropna(inplace = True, axis=1)

    df["Year"] = x

    df_list.append(df[["Democratic", "Republican", "Total Votes Cast", "Year"]])
    res = pd.concat(df_list)

    res["Republican Vote Share"] = res["Republican"]/res["Total Votes Cast"]

g1 = res[res.index=="Accomack County"].plot(x="Year", y="Republican Vote Share")
g1.get_figure().savefig('accomack.png')

    